In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, TFGPT2Model, TFGPT2ForSequenceClassification
import tensorflow as tf
import re
import os
!pip install tensorflow
from tensorflow.keras.metrics import AUC

In [3]:
os.environ['TF_ENABLE_EAGER_CLIENT_STREAMING_ENQUEUE'] = 'False'

In [4]:
democrat_file = "/content/drive/MyDrive/mount/democrat.csv"
republican_file = "/content/drive/MyDrive/mount/republican.csv"

model_name = "gpt2"
model_dir = "/content/drive/MyDrive/mount/results/dem_rep_model"

batch_size = 8
batch_size_fitting = 1024

In [5]:
democrat_df = pd.read_csv(democrat_file)
republican_df = pd.read_csv(republican_file)

In [6]:
democrat_df["label"] = 1
republican_df["label"] = 0

amount_of_entries = min(len(democrat_df), len(republican_df)) - 10

democrat_df = democrat_df.head(amount_of_entries)
republican_df = republican_df.head(amount_of_entries)

In [7]:
combined_df = pd.concat([democrat_df, republican_df], ignore_index=True)
combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    combined_df["text"].values,
    combined_df["label"].values,
    test_size=0.15,
    random_state=42
)

In [8]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [9]:
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
with strategy.scope():
    train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=512, return_tensors='tf')
    test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=512, return_tensors='tf')

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

In [ ]:
class GPT2Classifier(tf.keras.Model):
    def __init__(self, num_classes, l2_lambda=0.01):
        super(GPT2Classifier, self).__init__()
        self.gpt2 = TFGPT2Model.from_pretrained(model_name)
        self.dropout = tf.keras.layers.Dropout(0.1)
        self.dense = tf.keras.layers.Dense(num_classes, activation='softmax',
                                           kernel_regularizer=tf.keras.regularizers.L2(l2_lambda))

    def call(self, inputs):
        outputs = self.gpt2(inputs)[0]
        pooled_output = tf.reduce_mean(outputs, axis=1)
        pooled_output = self.dropout(pooled_output)
        logits = self.dense(pooled_output)
        return logits

In [ ]:
auc_metric = AUC()

In [ ]:
with strategy.scope():
    model = GPT2Classifier(num_classes=2)
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5) #orig 3e-5
    loss = tf.keras.losses.SparseCategoricalCrossentropy()
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2Model.

All the weights of TFGPT2Model were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


In [ ]:
i = 0

final_dataset = train_dataset.shuffle(amount_of_entries).batch(batch_size_fitting)
amount_of_steps = len(final_dataset)

for batch in final_dataset:
    print(f"Run {i + 1} out of {amount_of_steps}")
    model.fit(batch[0], batch[1], epochs=6, batch_size=batch_size)
    if i % 50 == 0:
        print(f"Saving at step {i}")
        model.save(model_dir)
        print("Saved")
    i += 1

In [ ]:
model.save(model_dir)

In [ ]:
with strategy.scope():
    from sklearn import metrics
    loss, accuracy, auc = model.evaluate(test_dataset.batch(batch_size))
    print(f"Test accuracy: {accuracy}")
    print(f"AUC: {auc}")

ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2066, in test_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2049, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2037, in run_step
        outputs = model.test_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1919, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 329, in __call__
        self._total_loss_mean.update_state(
    File "/usr/local/lib/python3

In [10]:
model = tf.keras.models.load_model(model_dir)

In [11]:
def predict_news_class(probabilities, threshold=0.5):
    republican_probability, democrat_probability = probabilities[0]
    if (republican_probability - democrat_probability)**2 > 0.3:
        if republican_probability > threshold:
            return "republican", republican_probability, democrat_probability, ((republican_probability - democrat_probability)**2)
        else:
            return "democrat", republican_probability, democrat_probability, ((republican_probability - democrat_probability)**2)
    else:
        return "neutral", republican_probability, democrat_probability, ((republican_probability - democrat_probability)**2)


In [12]:
def remove_emojis_and_links(text):
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s]|RT', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.replace("\n", "")
    return text.strip()

In [ ]:
while True:
    inputs = tokenizer(remove_emojis_and_links(input()), return_tensors='tf', max_length=512, truncation=True, padding='max_length')
    output = model(inputs)
    print(predict_news_class(output), sep = "\n")

('democrat', <tf.Tensor: shape=(), dtype=float32, numpy=0.05451727>, <tf.Tensor: shape=(), dtype=float32, numpy=0.9454827>, <tf.Tensor: shape=(), dtype=float32, numpy=0.79381937>)


In [ ]:
new_model.save("/content/drive/MyDrive/mount/results/model.tf")

In [ ]:
tf. __version__

'2.15.0'

In [ ]:
with strategy.scope():
    predictions = model.predict(test_dataset.batch(batch_size))

1378/1378 [==============================] - 778s 544ms/step


In [ ]:
from sklearn import metrics

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(test_labels, predictions[:, 1])

In [ ]:
roc_auc = auc_metric(fpr, tpr)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=10, fontname='Times New Roman')
plt.ylabel('True Positive Rate', fontsize=10, fontname='Times New Roman')
plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=12, fontname='Times New Roman')
plt.legend(loc="lower right", prop={'size': 10, 'family': 'Times New Roman'})
plt.xticks(fontsize=10, fontname='Times New Roman')
plt.yticks(fontsize=10, fontname='Times New Roman')
plt.show()